This is a simple example of SimSwap on swapping one single face within one image or one video. You can change the codes for inference based on other scripts for image or single face swapping.

Code path: https://github.com/neuralchen/SimSwap

Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

## Installation

All file changes made by this notebook are temporary. 
You can try to mount your own google drive to store files if you want.


In [ ]:
%cd /content
!git clone https://github.com/jrtaloma/SimSwap
!cd SimSwap && git pull

In [ ]:
%cd /content
!git clone https://github.com/yangxy/GPEN

# remove the example input
%rm -rf /content/GPEN/examples
%mkdir -p /content/GPEN/examples/imgs
%mkdir -p /content/GPEN/examples/grays

%cd GPEN
# set up the environment 
!pip install -r requirements.txt

!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

In [ ]:
import os
os.chdir('/content/SimSwap')

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

In [ ]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

### Now onedrive file can be downloaded in Colab directly!
### If the link blow is not permanent, you can just download it from the 
### open url(can be found at [our repo]/doc/guidance/preparation.md) and copy the assigned download link here.
### many thanks to woctezuma for this very useful help
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/


# Inference

In [ ]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

## Swap single within one image

In [ ]:
!wget -P /content https://static01.nyt.com/images/2017/12/05/watching/the-crown-watching-slide-NWN3/the-crown-watching-slide-NWN3-jumbo-v3.jpg?quality=90&auto=webp
!wget -P /content https://www.cyclinguk.org/sites/default/files/styles/crop_square/public/media-skyfish/298246/27_Lizzie_Deignan.jpg?itok=r7wM7B69&c=1bd5cf0369c2ef4491f9f01461ebbbbd

In [ ]:
import subprocess
TARGET = '/content/the-crown-watching-slide-NWN3-jumbo-v3.jpg?quality=90'
SOURCE = '/content/27_Lizzie_Deignan.jpg?itok=r7wM7B69'
subprocess.run(['cp', TARGET, '/content/target.jpg'])
subprocess.run(['cp', SOURCE, '/content/source.jpg'])

In [ ]:
!python test_wholeimage_swapsingle.py --isTrain false --use_mask --no_simswaplogo --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path /content/source.jpg --pic_b_path /content/target.jpg --output_path /content/swapA.jpg
!python test_wholeimage_swapsingle.py --isTrain false --use_mask --no_simswaplogo --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path /content/target.jpg --pic_b_path /content/source.jpg --output_path /content/swapB.jpg

In [23]:
#!rm /content/*.jpg

## GPEN
Face enhancement from [https://github.com/woctezuma/SimSwap-colab](https://github.com/woctezuma/SimSwap-colab)

It applies ***GAN Prior Embedded Network for Blind Face Restoration in the Wild***: [https://arxiv.org/abs/2105.06070](https://arxiv.org/abs/2105.06070)


In [ ]:
# download pre-trained models
!wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116085&Signature=GlUNW6%2B8FxvxWmE9jKIZYOOciKQ%3D" -O /content/GPEN/weights/RetinaFace-R50.pth
!wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-512.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116208&Signature=hBgvVvKVSNGeXqT8glG%2Bd2t2OKc%3D" -O /content/GPEN/weights/GPEN-BFR-512.pth
!wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-BFR-256.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116259&Signature=kMGJLSHqnvzzzqwtjUVBgngzX2s%3D" -O /content/GPEN/weights/GPEN-BFR-256.pth
!wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/realesrnet_x2.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1962694780&Signature=lI%2FolhA%2FyigiTRvoDIVbtMIyhjI%3D" -O /content/GPEN/weights/realesrnet_x2.pth
!wget "https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/ParseNet-latest.pth?OSSAccessKeyId=LTAI4G6bfnyW4TA4wFUXTYBe&Expires=1961116134&Signature=bnMwU1JogmNbARto6G%2B7iaJQCHs%3D" -O /content/GPEN/weights/ParseNet-latest.pth

In [16]:
# place images in GPEN/examples/imgs
%cp /content/swapA.jpg /content/GPEN/examples/imgs/
%cp /content/swapB.jpg /content/GPEN/examples/imgs/

In [15]:
os.chdir('/content/GPEN')

In [ ]:
# enhance faces
import os
import cv2
import argparse
import numpy as np
import matplotlib.pyplot as plt

import __init_paths
from face_enhancement import FaceEnhancement


class AttrDict(dict):
  def __init__(self, *args, **kwargs):
    super(AttrDict, self).__init__(*args, **kwargs)
    self.__dict__ = self


if __name__=="__main__":

    #model = {'name':'GPEN-BFR-512', 'sr_model':'realesrnet', 'sr_scale': 2, 'size':512, 'channel_multiplier':2, 'narrow':1}
    #model = {'name':'GPEN-BFR-256', 'sr_model':'realesrnet', 'size':256, 'channel_multiplier':1, 'narrow':0.5}

    args = AttrDict({'model':'GPEN-BFR-512',
            'task':'FaceEnhancement',
            'key':None,
            'in_size':512,
            'out_size':None,
            'channel_multiplier':2,
            'narrow':1,
            'alpha':1,
            'use_sr':True,
            'use_cuda':False,
            'save_face':True,
            'aligned':True,
            'sr_model':'realesrnet',
            'sr_scale':2,
            'tile_size':0,
            'indir':'examples/imgs',
            'outdir':'results/outs-bfr',
            'ext':'.jpg'}
    )

    os.makedirs(args.outdir, exist_ok=True)

    faceenhancer = FaceEnhancement(args, in_size=args.in_size, model=args.model, use_sr=args.use_sr, device='cuda' if args.use_cuda else 'cpu')

    file = os.path.join(args.indir, 'swapA.jpg')
    filename = os.path.basename(file)
    img = cv2.imread(file, cv2.IMREAD_COLOR) # BGR
    #im = cv2.resize(im, (0,0), fx=2, fy=2) #optional

    img_out, orig_faces, enhanced_faces = faceenhancer.process(img, aligned=False)
    
    img = cv2.resize(img, img_out.shape[:2][::-1])
    cv2.imwrite(os.path.join(args.outdir, '.'.join(filename.split('.')[:-1])+'_COMP.jpg'), np.hstack((img, img_out)))
    cv2.imwrite(os.path.join(args.outdir, '.'.join(filename.split('.')[:-1])+'_GPEN.jpg'), img_out)
        
    for m, (ef, of) in enumerate(zip(enhanced_faces, orig_faces)):
        of = cv2.resize(of, ef.shape[:2])
        cv2.imwrite(os.path.join(args.outdir, '.'.join(filename.split('.')[:-1])+'_face%02d'%m+'.jpg'), np.hstack((of, ef)))

In [22]:
#!rm -rf /content/GPEN/results/outs-bfr/* /content/GPEN/examples/imgs/*

## Swap single within one video

In [19]:
os.chdir('/content/SimSwap')

In [ ]:
import subprocess
TARGET = '/content/SimSwap/demo_file/multi_people_1080p.mp4'
SOURCE = '/content/SimSwap/demo_file/Iron_man.jpg'
subprocess.run(['cp', TARGET, '/content/target.mp4'])
subprocess.run(['cp', SOURCE, '/content/source.jpg'])
!python test_video_swapsingle.py --isTrain false --use_mask --no_simswaplogo --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path /content/source.jpg --video_path /content/target.mp4 --output_path /content/multi_test_swapsingle.mp4 --temp_path /content/temp_results

In [21]:
#!rm -rf /content/temp_results/*.jpg